In [ ]:
import numpy as np # linear b
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)b
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
train_path = '../input/the-great-indian-hiring-hackathon/Participants_Data_TGIH'
train = pd.read_csv('../input/the-great-indian-hiring-hackathon/Participants_Data_TGIH/Train.csv')
test =  pd.read_csv('../input/the-great-indian-hiring-hackathon/Participants_Data_TGIH/Test.csv')
sample_submission = pd.read_csv( '../input/the-great-indian-hiring-hackathon/Participants_Data_TGIH/Sample Submission.csv')


In [ ]:
train.head()

In [ ]:
test.head()

## Description of Dataset

- Invoice No - Invoice ID, encoded as Label
- StockCode - Unique code per stock, encoded as Label
- Description - The Description, encoded as Label
- Quantity - Quantity purchased
- InvoiceDate - Date of purchase
- UnitPrice - The target value, price of every product
- CustomerID - Unique Identifier for every Customer
- Country - Country of sales, encoded as Label

## No Of Records in the dataset

In [ ]:
train.info()

In [ ]:
test.info()

## get min max std deviation of the dataset /check quantiles

In [ ]:
train.describe()

In [ ]:
test.describe()

In [ ]:
train.nunique()

In [ ]:
# !pip install reg_resampler # will use if needed

In [ ]:
Target_col = 'UnitPrice'

In [ ]:
categorical_vars = ['InvoiceNo', 'StockCode', 'Description', 'CustomerID', 'Country']
conts = ['UnitPrice', 'Quantity']

In [ ]:
train.Quantity.plot.hist()

In [ ]:
train.UnitPrice.plot.hist()

In [ ]:
### By observing test dataset, Observed that test dont have quantity values higher than 4800 and lower than 9360
### Also the UnitPrice column have 80000 which is an outlier i consider

In [ ]:
# outliers
train = train[train['Quantity']<=4800]
train = train[train['Quantity']>-9360.000000]
# Removed outliers
train = train[train['UnitPrice']<30000]

In [ ]:
for i in categorical_vars:
    print(train.groupby(i)['UnitPrice'].value_counts().nlargest(5))

## Feature Engeneering

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(train['Country'])
train['Country_encode'] = le.transform(train['Country'])
test['Country_encode'] = le.transform(test['Country'])

In [ ]:
# performing freq encoding

dicto = train['Country'].value_counts(normalize=True).to_dict()
train['Country'] = train['Country'].map(dicto)
dicto = test['Country'].value_counts(normalize=True).to_dict()
test['Country'] = test['Country'].map(dicto)

In [ ]:
dicto = train['StockCode'].value_counts(normalize=True).to_dict()
train['StockCode_freq'] = train['StockCode'].map(dicto)
dicto = test['StockCode'].value_counts(normalize=True).to_dict()
test['StockCode_freq'] = test['StockCode'].map(dicto)

In [ ]:
from scipy import stats

In [ ]:
fitted_data, fitted_lambda = stats.boxcox(np.abs(train['Quantity']))
train['Quantity_box'] = fitted_data
fitted_data, fitted_lambda = stats.boxcox(np.abs(test['Quantity']))
test['Quantity_box'] = fitted_data

In [ ]:
sns.distplot(fitted_data)

In [ ]:
# date extractor info
train['InvoiceDate'] = pd.to_datetime(train['InvoiceDate'])
test['InvoiceDate'] = pd.to_datetime(test['InvoiceDate'])

In [ ]:
for df in [train, test]:
    df['year'] = df['InvoiceDate'].dt.year
    df['day'] = df['InvoiceDate'].dt.day
    df['hour'] = df['InvoiceDate'].dt.hour
    df['minutes'] = df['InvoiceDate'].dt.minute
    df['day_of_year'] = df['InvoiceDate'].dt.dayofyear
    df['day_of_year'] = df['InvoiceDate'].dt.dayofyear
    df['day_of_week'] = df['InvoiceDate'].dt.dayofweek
    df['ordinals'] = train['InvoiceDate'].apply(lambda x:x.toordinal()/70000)

In [ ]:
train.head()

In [ ]:
from sklearn.preprocessing import KBinsDiscretizer
est = KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='quantile')
est.fit(train['UnitPrice'].values.reshape(-1,1))
train['formed'] = est.transform(train['UnitPrice'].values.reshape(-1,1)).astype(int)

In [ ]:
from sklearn.preprocessing import KBinsDiscretizer
est = KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='quantile')
est.fit(train['Quantity'].values.reshape(-1,1))
train['Quantity_quantiles'] = est.transform(train['Quantity'].values.reshape(-1,1)).astype(int)
test['Quantity_quantiles'] = est.transform(test['Quantity'].values.reshape(-1,1)).astype(int)

In [ ]:
train['log_UnitPrice'] = np.log1p(train['UnitPrice'])

In [ ]:
train['CustomerID'] = train['CustomerID'].astype('int')
test['CustomerID'] = test['CustomerID'].astype('int')

In [ ]:
train['InvoiceDate'].head()

In [ ]:
## Modelling part

In [ ]:
X = train.drop(['InvoiceNo', 'InvoiceDate','UnitPrice', 'StockCode_freq', 'log_UnitPrice', 'formed', 'year','minutes','hour', 'Quantity',
               'Quantity_quantiles', 'Country_encode', 'StockCode_freq'], axis=1).values
y = train['UnitPrice'].values
strat_y = train['formed']
X_test = test.drop(['InvoiceNo', 'InvoiceDate', 'year','minutes','hour', 'Quantity',
               'Quantity_quantiles',  'Country_encode','StockCode_freq'], axis=1).values

### Features selected for training

In [ ]:
train.drop(['InvoiceNo', 'UnitPrice', 'InvoiceDate', 'log_UnitPrice', 'formed', 'year','minutes','hour','Quantity',
               'Quantity_quantiles', 'Country_encode','StockCode_freq'], axis=1).columns

In [ ]:
from sklearn.linear_model import Lasso, LinearRegression,Ridge
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import SVR

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt

In [ ]:
from sklearn.model_selection import StratifiedKFold,KFold

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import ExtraTreeRegressor
import xgboost as xgb
import lightgbm as lgb
from sklearn import linear_model
from catboost import CatBoostRegressor

### LGBM

In [ ]:
predictions = np.zeros(test.shape[0])
oobs = np.zeros(train.shape[0])
y = train['UnitPrice'].values
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
kf=KFold(n_splits=10, shuffle=True, random_state=42)
params = {'n_estimators':100000, 'n_jobs':-1, 'random_state':42}
model = lgb.LGBMRegressor(**params)

for i, (train_id, valid_id) in enumerate(kf.split(X,y)):
    print("fold ", i)  
    X_train, y_train = X[train_id], y[train_id]
    X_valid, y_valid = X[valid_id], y[valid_id]
# # lambda y_true, y_pred:[rmse(y_true,y_pred)]
    model.fit(X_train, y_train, eval_set =[(X_valid, y_valid)],  early_stopping_rounds=200, verbose=1000, eval_metric='rmse')

    oobs[valid_id] = model.predict(X_valid)
    predictions += model.predict(X_test)
rmse = sqrt(mean_squared_error(y,oobs))
finals = predictions/10
finals[finals<0] = 0
lgb_predictions = finals
sample_submission['UnitPrice'] = finals
sample_submission.to_csv('lgb_final.csv', index=False)
print("RMSE", rmse)
lgb_oobs = oobs
print("best oob catboost")
pd.DataFrame(data=oobs).to_csv('oob_lgb.csv')

### CatBoost

In [ ]:
predictions = np.zeros(test.shape[0])
oobs = np.zeros(train.shape[0])
y = train['UnitPrice'].values
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
kf=KFold(n_splits=10, shuffle=True, random_state=42)
params = {'n_estimators':50000, 'learning_rate': 0.1, 'eval_metric':'RMSE', 'loss_function':'RMSE', 'task_type':'GPU', 'devices':'0:1', 'random_state':42}
model = CatBoostRegressor(**params)

for i, (train_id, valid_id) in enumerate(kf.split(X,y)):
    print("fold ", i)

    X_train, y_train = X[train_id], y[train_id]
    X_valid, y_valid = X[valid_id], y[valid_id]

    model.fit(X_train, y_train, eval_set =[(X_valid, y_valid)],  early_stopping_rounds=200, verbose=1000)

    oobs[valid_id] = model.predict(X_valid)
    predictions += model.predict(X_test)

rmse = sqrt(mean_squared_error(y, oobs))
finals = predictions/10
finals[finals<0] = 0
catboost_predictions = finals
sample_submission['UnitPrice'] = finals
sample_submission.to_csv('catboost_final.csv', index=False)
print(rmse)

In [ ]:
catboost_oobs = oobs
print("best oob catboost")
pd.DataFrame(data=oobs).to_csv('oob_catboost.csv')

### After analysis, I found that this weights are best for submission

In [ ]:
CV = (lgb_predictions*0.7)+(catboost_predictions*0.3)

In [ ]:
sample_submission['UnitPrice'] = CV
sample_submission.to_csv('combined_2_weights_CV_2.csv', index=False)

## Training model on all the training set at once for xgb and catboost

In [ ]:
params = {'n_estimators':10000, 'n_jobs':-1, 'random_state':42}
model = lgb.LGBMRegressor(**params)
model.fit(X,y)
predictions_lgb = model.predict(X_test)

In [ ]:
sample_submission['UnitPrice'] = predictions_lgb
sample_submission.to_csv('LGB_PLB.csv', index=False)
sample_submission['UnitPrice'] = (predictions_lgb*0.3)+(lgb_predictions*0.7)
sample_submission.to_csv('LGB_PLB_CV.csv', index=False)

In [ ]:
params = {'n_estimators':10000, 'learning_rate': 0.1, 'eval_metric':'RMSE', 'loss_function':'RMSE', 'task_type':'GPU', 'devices':'0:1', 'random_state':42}
model = CatBoostRegressor(**params)
model.fit(X,y,verbose=1000)
predictions_catboost = model.predict(X_test)

In [ ]:
PLB = (predictions_lgb*0.7)+(predictions_catboost*0.3)

In [ ]:
sample_submission['UnitPrice'] = (CV*0.7) + (PLB*0.3)
sample_submission.to_csv('combined_2_PLB_and_CV.csv', index=False)

## some of stockcode had UnitPrice repating so used those value for mapping

In [ ]:
train.sort_values(['StockCode', 'UnitPrice', 'InvoiceDate'])[['StockCode', 'UnitPrice', 'InvoiceDate']]

In [ ]:
train = train.sort_values(['StockCode', 'UnitPrice', 'InvoiceDate'])
analysis = train.groupby(['StockCode', 'UnitPrice']).count().reset_index()
ana = analysis.loc[~analysis.StockCode.isin(analysis[analysis.duplicated(['StockCode'])].StockCode.unique().tolist()),['StockCode', 'UnitPrice']]
ana['StockCode'] = ana['StockCode'].astype('str')
ana.set_index('StockCode', inplace=True)

# only LGB gave public lb of 23.37 and private lb 28.33564 (Machine Hack Considered this as final since it was best public lb)
# catboost and lgb gave public lb of 23.84114 around 28.09341